In [1]:
%run widgets.py

In [6]:
s = Segmentation(
    'tls_park', 
    training_repo='/data1/sat/dataset_tls_park_444/train.txt',
    testing_repo='/data1/sat/dataset_tls_park_444/test.txt',
    host='10.10.77.61',
    port=8500,
    #port=8400,
    img_height=480,
    img_width=480,
    model_repo='/data1/sat/xolive/models/model_tls_park_debug',
    nclasses=2,
    #template='deeplab_vgg16',
    iterations=15000,
    test_interval=500,
    snapshot_interval=1000,
    batch_size=5,
    test_batch_size=1,
    noise_prob=0.001,
    class_weights=[0.1, 1.0],
    model_postfix='test_park_xo3',
    weights='/data1/segmentation/vgg16_init_deeplab.caffemodel',
    gpuid=2
)#.run()
s

In [7]:
c = Classification(
    'xo_bbc_5_classes', 
    training_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/train/',
    testing_repo='/data1/xolive/bbc_sounds/sounds_dataset_1.0/test/',
    host='10.10.77.61',
    port=8500,
    model_repo='/data1/bbc_sounds/bbc_5_classes',
    layers='["1CR32", "1CR64", "1CR128", "1024"]',
    template='convnet',
    gpuid=1,
    img_width=257,
    img_height=257
)
c
# TODO unchanged data

In [4]:
c = Classification(
    'dogs_cats', 
    training_repo='/data1/kaggle/dogs_cats/train/',
    tsplit=.2,
    host='10.10.77.61',
    port=8500,
    model_repo='/data1/xolive/models/dogs_cats',
    template='convnet',
    gpuid=0,
    layers=['1CR32','1CR64','1CR128','1024'],
    img_width=150,
    img_height=150,
    mirror=True,
    rotate=True,
)
c
# TODO unchanged data

In [30]:
d = Detection(
    "eoliennes",
    training_repo= "/data1/sat/dataset_windturbine_europe_16_test/train.txt",
    testing_repo= "/data1/sat/dataset_windturbine_europe_16_test/test.txt",
    host='10.10.77.61',
    port=8500,
    model_repo='/data1/xolive/models/detection',
    gpuid=0,
    img_width=300,
    img_height=300,
    snapshot_interval=1000,
    iterations=5000,
    template="ssd_300",
    mirror=True,
    rotate=True, # no default
    finetune=True,
    weights="/data1/detection/VGG_ILSVRC_16_layers_fc_reduced.caffemodel",
    batch_size=24,
    test_batch_size=4,
    nclasses=2,
)
d

In [31]:
c = CSV(
    'forest',
    host='10.10.77.61',
    port=8500,
    training_repo="/data1/xolive/forest_type/train.csv",
    model_repo="/data1/xolive/models/forest",
    csv_label='Cover_Type',
    csv_id='Id',
    iterations=10000,
)
c

In [32]:
t = Text(
    'news',         
    training_repo = '/data1/xolive/news/news20',
    testing_repo=None,
    host='10.10.77.61',
    port=8500,
    model_repo="/data1/xolive/models/news",
    nclasses=20,
    gpuid=1
)
t

In [2]:
class OCR(MLWidget, ImageTrainerMixin):
    
    ctc = True

    def update_train_file_list(self, *args):
        with self.output:
            # print (Path(self.training_repo.value).read_text().split('\n'))
            self.file_dict = {
                Path(x.split()[0]): x.split()[1:]
                for x in Path(self.training_repo.value).read_text().split("\n")
                if len(x.split()) >= 2
            }

            self.file_list.options = [
                fh.as_posix()
                for fh in sample_from_iterable(self.file_dict.keys(), 10)
            ]

    def update_test_file_list(self, *args):
        with self.output:
            # print (Path(self.training_repo.value).read_text().split('\n'))
            self.file_dict = {
                Path(x.split()[0]): x.split()[1:]
                for x in Path(self.testing_repo.value).read_text().split("\n")
                if len(x.split()) >= 2
            }

            self.file_list.options = [
                fh.as_posix()
                for fh in sample_from_iterable(self.file_dict.keys(), 10)
            ]
            
    def display_img(self, args):
        self.output.clear_output()
        with self.output:
            for path in args["new"]:
                shape, img = img_handle(Path(path))
                if self.img_width.value == "":
                    self.img_width.value = str(shape[0])
                if self.img_height.value == "":
                    self.img_height.value = str(shape[1])
                display(
                    img
                )  # TODO display next to each other with shape info as well
                print(" ".join(o.file_dict[Path(path)]))

    def __init__(
        self,
        sname: str,
        *,  # unnamed parameters are forbidden
        training_repo: Path = None,
        testing_repo: Path = None,
        host: str = "localhost",
        port: int = 1234,
        path: str = "",
        nclasses: int = -1,
        description: str = "OCR service",
        model_repo: Optional[str] = None,
        img_width: Optional[int] = None,
        img_height: Optional[int] = None,
        base_lr: float = 1e-4,
        iterations: int = 10000,
        snapshot_interval: int = 5000,
        test_interval: int = 1000,
        gpuid: int = 0,
        layers: List[str] = [],
        template: Optional[str] = None,
        mirror: bool = False,
        rotate: bool = False,
        tsplit: float = 0.0,
        finetune: bool = False,
        resume: bool = False,
        bw: bool = False,
        crop_size: int = -1,
        batch_size: int = 32,
        test_batch_size: int = 16,
        iter_size: int = 1,
        solver_type: Solver = "SGD",
        noise_prob: float = 0.0,
        distort_prob: float = 0.0,
        test_init: bool = False,
        class_weights: List[float] = [],
        weights: Path = None,
        tboard: Optional[Path] = None,
        ignore_label: int = -1,
        multi_label: bool = False,
        regression: bool = False,
        rand_skip: int = 0,
        timesteps: int = 32,
        unchanged_data: bool = False,
        target_repository: str = ""
    ) -> None:

        super().__init__(sname, locals())

        self.train_labels = Button(
            description=Path(self.training_repo.value).name
        )
        self.test_labels = Button(
            description=Path(self.testing_repo.value).name
        )

        # self.testing_repo.observe(self.update_test_button, names="value")
        # self.training_repo.observe(self.update_train_button, names="value")

        self.train_labels.on_click(self.update_train_file_list)
        self.test_labels.on_click(self.update_test_file_list)

        
        self.file_list.observe(self.display_img, names="value")

        self._img_explorer.children = [
            HBox([HBox([self.train_labels, self.test_labels])]),
            self.file_list,
            self.output,
        ]

        #self.update_label_list(())


In [9]:
!head -n 100 /data1/ocr/synthtext/imglist_ocr.txt > /data1/xolive/ocr_train.txt

In [11]:
!tail -n 100 /data1/ocr/synthtext/imglist_ocr.txt > /data1/xolive/ocr_test.txt

In [12]:
o = OCR('synthtxt',
    training_repo = '/data1/xolive/ocr_train.txt',
    testing_repo='/data1/xolive/ocr_test.txt',
    host='10.10.77.61',
    port=8500,
    model_repo="/data1/xolive/models/synthtxt",
        img_width=220,
        img_height=80,
        nclasses= 37,
        template='crnn',
        solver_type='AMSGRAD'
   )
o

In [35]:
"abcdefghijklmnopqrstuvwxyz0123456789,;.!?:’“/\_@#$%^&*~`+-=<>()[]{}"

'abcdefghijklmnopqrstuvwxyz0123456789,;.!?:’“/\\_@#$%^&*~`+-=<>()[]{}'

In [36]:
len(_)

67

In [5]:
!head -n 100 /data1/ocr/synthtext/imglist_ocr.txt > /data1/xolive/ocr_test.txt